In [2]:
import numpy as np 
import time
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
import requests 
from pandas.io.json import json_normalize 
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 
import folium 

Solving environment: done

# All requested packages already installed.



# 1.Introduction Section:
### Discussion of the business problem and the audience who would be interested in this project-
### Description of the Problem and Background:-

#### Scenario:
I am a data scientist and lets assume I am residing in Downtown Singapore. I currently live within walking distance to Downtown Telok Ayer MRT metro station and I enjoy many ammenities and venues in the area, such as various international cousine restaurants, cafes, food shops and entertainment. I have been offered a great opportunity to work for a leader firm in Manhattan, NY. I am very excited and I want to use this opportunity to practice my learnings in Coursera in order to answer relevant questions arisen. The key question is : How can I find a convenient and enjoyable place similar to mine now in Singapore? Certainly, I can use available real estate apps and Google but the idea is to use and apply myself the learned tools during the course. In order to make a comparison and evaluation of the rental options in Manhattan NY, I must set some basis, therefore the apartment in Manhattan must meet the following demands:

-Apartment must be 2 or 3 bedrooms                                                   
-Desired location is near a metro station                
-Price of rent not exceed $7,000 per month                 
-Top ammenities in the selected neighborhood shall be similar to current residence                       
-Desirable to have venues such as coffee shops, restaurants Asian Thai, wine stores, gym and food shops                       
-As a reference, I have included a map of venues near current residence in Singapore.              

#### Business Problem:
The challenge is to find a suitable apartment for rent in Manhattan NY that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.

#### Interested Audience:
I believe this is a relevant challenge with valid questions for anyone moving to other large city in US or Asia. The same methodology can be applied in accordance to demands as applicable. This case is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.

# 2. Data Section:

## Description of the data and its sources that will be used to solve the problem-

### Description of the Data:-
The following data is required to answer the issues of the problem:-

-List of Boroughs and neighborhoods of Manhattan with their geodata (latitud and longitud)            
-List of Subway metro stations in Manhattan with their address location                    
-List of apartments for rent in Manhattan area with their addresses and price                      
-Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc                  
-Venues for each Manhattan neighborhood ( than can be clustered)                  
-Venues for subway metro stations, as needed.          

### How the data will be used to solve the problem:-

The data will be used as follows:               

-Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)                     
-Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately                               
-Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.                             
-create a map that depicts, for instance, the average rental price per square ft.
-I will be able to quickly point to the popups to know the relative price per subway area.                         
-Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.                     






## Reference of venues around current residence in Singapore for comparison to Manhattan place

In [4]:
address = 'Mccallum Street, Singapore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore home are {}, {}.'.format(latitude, longitude))


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Singapore home are 1.2784801, 103.8493717.


In [27]:
neighborhood_latitude=1.2784801
neighborhood_longitude=103.8493717

In [29]:
LIMIT = 100 
radius = 500
CLIENT_ID = '2YT42EKWBB3MT5SN3S1JTTWMVGPDBOAV5BI2RSMZ5ARH45ZW' 
CLIENT_SECRET = 'G0OZOTD5ILIPWA4JY2VKAKI3UZP2EGDBGCQWKZ1OJDJMPGT2'
VERSION = '20180604'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=2YT42EKWBB3MT5SN3S1JTTWMVGPDBOAV5BI2RSMZ5ARH45ZW&client_secret=G0OZOTD5ILIPWA4JY2VKAKI3UZP2EGDBGCQWKZ1OJDJMPGT2&v=20180604&ll=1.2784801,103.8493717&radius=500&limit=100'

In [30]:
results = requests.get(url).json()

In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]
SGnearby_venues.head(10)

,name,categories,lat,lng
0,The Westin Singapore,Hotel,1.278275,103.850772
1,Pure Fitness,Gym,1.278631,103.851487
2,Lobby Lounge Westin,Bar,1.277811,103.850966
3,Mellower Coffee,Café,1.277814,103.848188
4,Lau Pa Sat Satay Street,Street Food Gathering,1.280261,103.850235
5,Napoleon Food & Wine Bar,Wine Bar,1.279925,103.847333
6,Westin Infinity Pool,Pool,1.278057,103.851077
7,Sofitel So Singapore,Hotel,1.280017,103.849813
8,Anglo Indian Cafe & Bar,Indian Restaurant,1.279084,103.850127
9,Asia Square,Plaza,1.278340,103.851513


In [34]:
map_sing = folium.Map(location=[latitude, longitude], zoom_start=20)
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sing)  
    
map_sing
